# entropía y unimorph!

## datos 

In [702]:
#f = open("datos/shp", "r")
#lemas = list(zip(*[line.split() for line in f]))[0]

In [703]:
#f = open("datos/shp", "r")
#inflexiones = list(zip(*[line.split() for line in f]))[1]

In [704]:
#f = open("datos/shp", "r")
#codes = list(zip(*[line.split() for line in f]))[2]

In [705]:
#lemas = [lemas[i] for i in range(len(lemas)) if i not in [4820,4821]]
#inflexiones = [inflexiones[i] for i in range(len(inflexiones)) if i not in [4820,4821]]
#codes = [codes[i] for i in range(len(codes)) if i not in [4820,4821]]

In [706]:
#shp = {'formas':lemas, 'inflexiones': inflexiones, 'codes':codes}

In [707]:
#datos = pd.DataFrame.from_dict(shp)

In [708]:
#datos

In [94]:
## https://github.com/unimorph/que

import pandas as pd

lengua = 'cni' ## (puede ser, dak, arn, cni, aym, mao y que)
datos = pd.read_csv('datos/'+lengua, sep='\t', header=None)
datos.columns = ['formas','inflexiones','codes']

In [95]:
## solo los verbos

datos = datos[datos['codes'].str.startswith('V')]

In [96]:
## nos quedamos solo con formas e inflexiones

datos = datos[['formas', 'inflexiones']]

In [97]:
datos

,formas,inflexiones
0,aabintaantsi,aabintaantsi
1,aabintaantsi,aabintaje
2,aabintaantsi,aabintaje
3,aabintaantsi,aabintaji
4,aabintaantsi,aabintaji
...,...,...
20065,yotaantsi,piyotake
20066,yotaantsi,piyotake
20067,yotaantsi,piyote
20068,yotaantsi,piyoti


In [98]:
lemas = list(datos['formas'])

In [99]:
inflexiones = list(datos['inflexiones'])

In [100]:
print(len(lemas))

5622


In [101]:
## Primero, definimos una lista de lemas únicos

lemas_unicos = []

## recorremos la lista de lemas
for lema in lemas:
    ## solo agregamos si no está en lemas_unicos
    if lema not in lemas_unicos:
        lemas_unicos+=[lema]

In [102]:
## definimos un diccionario vacío

D_lema_inflex = {}

## recorremos la lista de lemas_unicos
for lema in lemas_unicos:
    ## agregamos en cada iteración, una lista vacía :)
    D_lema_inflex[lema]=[]

In [103]:
## recorremos la lista de lemas (la original). Usamos los índices!

for i in range(len(lemas)):
    ## accedemos al lema de la posición i
    lema = lemas[i]
    ## guardamos en la lista asociada las inflexiones del lema (que están en la lista inflexiones)
    D_lema_inflex[lema]+=[inflexiones[i]]

In [104]:
len(D_lema_inflex)

150

## muestreo de lemas/inflexiones

In [105]:
import random

lista_lemas = []
lista_inflexiones = []

s = 100
for r in range(100):
    n = len(D_lema_inflex)
    if n < s:
        indexes = random.sample(list(range(len(D_lema_inflex.keys()))),n)
    else:
        indexes = random.sample(list(range(len(D_lema_inflex.keys()))),s)
    lemas = [list(D_lema_inflex.keys())[i] for i in indexes]
    inflexiones = []
    for l in lemas:
        inflexiones += D_lema_inflex[l]
    lista_lemas += [lemas]
    lista_inflexiones += [inflexiones]

## entropías

In [106]:
lista_strings = []

for i in range(len(lista_lemas)):
    
    ## lemas/inflexiones
    lemas = lista_lemas[i]
    inflexiones = lista_inflexiones[i]
    
    STRINGS = {'lemas': [], 'inflexiones': []}
    lem = ['#'+s+'$' for s in lemas]
    inf = ['#'+s+'$' for s in inflexiones]
    STRINGS['lemas'] = lem
    STRINGS['inflexiones'] = inf
    lista_strings += [STRINGS]

In [107]:
def find_ngrams(input_list, n):
    return [''.join(tup) for tup in list(zip(*[input_list[i:] for i in range(n)]))]

In [108]:
find_ngrams('#achalachichunku$',3)

['#ac',
 'ach',
 'cha',
 'hal',
 'ala',
 'lac',
 'ach',
 'chi',
 'hic',
 'ich',
 'chu',
 'hun',
 'unk',
 'nku',
 'ku$']

In [109]:
lista_ngramas = []

for item in lista_strings:
    
    STRINGS = item
    
    NGRAMAS = {'lemas': [], 'inflexiones': []}

    for w in STRINGS['lemas']:
        NGRAMAS['lemas']+=find_ngrams(w,3)
    
    for w in STRINGS['inflexiones']:
        NGRAMAS['inflexiones']+=find_ngrams(w,3)
    lista_ngramas += [NGRAMAS]

In [110]:
from collections import Counter
from scipy.stats import entropy
import math

lista_entropias = []

for item in lista_ngramas:
    NGRAMAS = item
    entropias = {}

    for key in NGRAMAS.keys():
        C = list(dict(Counter(NGRAMAS[key])).values())
        H = entropy(C,base = sum(C))
        entropias[key] = H
    lista_entropias += [entropias]

In [111]:
import numpy as np

lista_entropias_lemas_mean = np.mean([D['lemas'] for D in lista_entropias])
lista_entropias_lemas_std = np.std([D['lemas'] for D in lista_entropias])

In [112]:
lista_entropias_lemas_mean, lista_entropias_lemas_std

(0.5963378998546542, 0.003964256921982297)

In [113]:
lista_entropias_inflex_mean = np.mean([D['inflexiones'] for D in lista_entropias])
lista_entropias_inflex_std = np.std([D['inflexiones'] for D in lista_entropias])

In [114]:
lista_entropias_inflex_mean, lista_entropias_inflex_std

(0.5177060389196284, 0.0024123925490850736)

In [115]:
lista_entropias_lemas_mean - lista_entropias_inflex_mean

0.07863186093502583

In [116]:
D_lema_inflex.keys()

dict_keys(['aabintaantsi', 'aantsi', 'abinata', 'abisaantsi', 'abitsanotaantsi', 'ajirikaantsi', 'akishitaantsi', 'amaantsi', 'amaataantsi', 'amajataantsi', 'amashaetaanti', 'amenaantsi', 'ametaantsi', 'amitakotaantsi', 'ampitsataantsi', 'aneenkaantsi', 'aniibeetaantsi', 'aniitaantsi', 'antaantsi', 'antabeetaantsi', 'antanataantsi', 'apajirikaantsi', 'apatotaantsi', 'apiitaantsi', 'araantsi', 'areetaantsi', 'asaankaantsi', 'asankaantsi', 'ashirejaantsi', 'ashitaantsi', 'ashitareantsi', 'ataitaantsi', 'atisankaantsi', 'atsikaantsi', 'atsipetaantsi', 'atsokijaantsi', 'atsotaantsi', 'bametaantsi', 'batsataantsi', 'betakaantsi', 'betsataantsi', 'betsikaantsi', 'chekaantsi', 'intaantsi', 'iraantsi', 'jaantsi', 'jataantsi', 'jibataantsi', 'jikotaantsi', 'jimatsitaantsi', 'jirotaantsi', 'kaataantsi', 'kabichoaantsi', 'kajataantsi', 'kajemaantsi', 'kamantaantsi', 'kametojaantsi', 'kantaantsi', 'karajaantsi', 'katiaantsi', 'katinkaantsi', 'katsiinkataantsi', 'kemaantsi', 'kemityonkaantsi', 'kem